# Import packages

In [ ]:
import os
import cv2
import pandas as pd
import cmaps
import cmocean
import numpy as np
import xarray as xr
import scipy.io as sio
import matplotlib.pyplot as plt
import matplotlib as mpl
import matplotlib.gridspec as gridspec
from netCDF4 import Dataset
from datetime import date
from mpl_toolkits.basemap import Basemap
from matplotlib.pyplot import Polygon
from matplotlib import rcParams
from matplotlib.backends.backend_pdf import PdfPages
version = mpl.__version__
rcParams['font.family'] = 'sans-serif'
directory   = '/srv/scratch/z3533156'

# Import time functions

In [ ]:
def datestring_to_serial_day(datestring,epochY=1990,epochm=1,epochd=1,epochH=0,epochM=0):
    import pandas as pd
    import datetime
    serial_day_timedelta = pd.to_datetime(datestring) - datetime.datetime(epochY,epochm,epochd,epochH,epochM)
    corrected_serial_day_number = serial_day_timedelta.days + serial_day_timedelta.seconds/86400
    return corrected_serial_day_number
def serial_day_to_datestring(day,epochY=1990,epochm=1,epochd=1,epochH=0,epochM=0):
    import datetime
    corrected_date = datetime.datetime(epochY,epochm,epochd,epochH,epochM) + datetime.timedelta(day)
    return corrected_date.strftime("%Y-%m-%d")  

# Read data

In [ ]:
AVISO_SLA       = Dataset(directory+'/MHW/AVISO/AVISO_EAC_MHW.nc','r')
SLA_daily       = np.transpose(AVISO_SLA.variables['sla'],[2,1,0])
SSH_daily       = np.transpose(AVISO_SLA.variables['adt'],[2,1,0])
lon0            = AVISO_SLA.variables['lon'][:]
lat0            = AVISO_SLA.variables['lat'][:]
lon             = np.tile(lon0,[np.size(lat0,0),1]).transpose()
lat             = np.tile(lat0,[np.size(lon0,0),1])
dataset         = sio.loadmat(directory+'/MHW/Figure_data/Figure1_OSTIA_MHW.mat')
daily_t         = np.arange(date(1993,1,1).toordinal(),date(2022,12,31).toordinal()+1,1)
daily_t1        = np.arange(date(2021,12,1).toordinal(),date(2022,2,28).toordinal()+1,1)
daily_dates     = [date.fromordinal(tt.astype(int)) for tt in daily_t1]
SLA             = SLA_daily[:,:,(daily_t>=date(2021,12,1).toordinal()) & (daily_t<=date(2022,2,28).toordinal())]
SSH             = SSH_daily[:,:,(daily_t>=date(2021,12,1).toordinal()) & (daily_t<=date(2022,2,28).toordinal())]
dataset         = sio.loadmat(directory+'/MHW/Figure_data/Figure6_SLA_Vtrans.mat')
lon0            = dataset['lon'][:,0]
lat0            = dataset['lat'][:,0]
SLA_Mmean       = dataset['SLA_meridional_mean'][:,:]
SLA_AVISO_ts    = dataset['SLA_AVISO_ts'][0,:]
SLA_PK_AVISO_ts = dataset['SLA_PK_AVISO_ts'][:,0]
SLA_PK_BRAN_ts  = dataset['SLA_PK_BRAN_ts'][:,0]
time_daily0     = dataset['time_daily'][:,0] + date(1950,1,1).toordinal()
time_daily      = [date.fromordinal(tt.astype(int)) for tt in time_daily0]
dates_daily     = np.tile(time_daily, (np.shape(lon0)[0],1))
lon_daily       = np.tile(lon0,(np.shape(dates_daily)[1],1)).transpose()
lat_daily       = np.tile(lat0,(np.shape(dates_daily)[1],1)).transpose()
# ################################################################################
PK_data      = pd.read_csv(directory+'/MHW/Figure_data/PK.csv')
DATE         = pd.date_range('30/11/2021 00:30:00', periods=132411, freq='T')
series       = pd.Series(PK_data['Tide'].values, index=DATE)
series1      = series.resample('1440T').mean()-0.916
PK_tide      = series1.values
daily_t2     = np.arange(date(2021,11,30).toordinal(),date(2022,3,1).toordinal()+1,1)
daily_dates2 = [date.fromordinal(tt.astype(int)) for tt in daily_t2]
#################################################################################
dataset1          = sio.loadmat(directory+'/MHW/Figure_data/Figure1_OSTIA_MHW.mat')
SST_daily_ts      = dataset1['SST_daily_ts'][0,-122:-30]
SST_daily_clim_ts = dataset1['SST_daily_clim_ts'][0,-122:-30]
SSTA_daily        = SST_daily_ts - SST_daily_clim_ts
#################################################################################
dataset2          = sio.loadmat(directory+'/MHW/Figure_data/Figure6_SLA_composite_extreme.mat')
SLA_MHW           = dataset2['SLA_MHW'][:,:]
SSH_MHW           = dataset2['SSH_MHW'][:,:]
SSH_mean          = dataset2['SSH_mean'][:,:]
SLA_MHW_pvalue    = dataset2['SLA_MHW_pvalue'][:,:]
#################################################################################
dataset3          = sio.loadmat(directory+'/MHW/Figure_data/Figure_etopo_AVISO.mat')
depth             = dataset3['depth'][:,:]
depth[lon>155]    = np.nan

# Plot the spatial distribution of sea level anomalies

In [ ]:
fig    = plt.figure(figsize=(24, 24))
gs     = gridspec.GridSpec(2,2)
padspacescale = 20
labelpadscale = 8
linefont      = 8
labelfont     = 30
scale1        = 1.0
m_scale       = 150
ssh_levels1   = np.linspace(0.949,0.951,2)
sla_levels1   = np.linspace(-0.001,0.001,2)
levels1       = np.linspace(-0.5,0.5,50)
levels2       = np.linspace(-0.2,0.2,50)
tick_marks1   = np.linspace(-0.5,0.5,11)
tick_marks2   = np.linspace(-0.2,0.2,5)
cmaps1        = cmaps.cmocean_balance
labels        = ['a','b','c']
##############################################################################################
ax1 = fig.add_subplot(gs[0])            
l, b, w, h = ax1.get_position().bounds
CB1 = plt.contourf(lon_daily,dates_daily,SLA_Mmean, cmap=cmaps1,levels=levels1,origin='lower',extend='both')
plt.fill_betweenx(time_daily[13:40], 144.875, 165.125, facecolor='xkcd:cool grey', alpha=0.4)
plt.fill_betweenx(time_daily[60:88], 144.875, 165.125, facecolor='xkcd:cool grey', alpha=0.4)
plt.xlim(lon_daily[23,0], lon_daily[-1,0])
plt.ylim(dates_daily[0,1], dates_daily[0,-1])
xminorLocator   = plt.MultipleLocator(1)
xmajorLocator   = plt.MultipleLocator(2)
xmajorFormatter = plt.FormatStrFormatter('%2.0f')
ax1.xaxis.set_minor_locator(xminorLocator)
ax1.xaxis.set_major_locator(xmajorLocator)
ax1.xaxis.set_major_formatter(xmajorFormatter)  
plt.xticks(fontsize=0.9*labelfont)
ax1.axes.xaxis.set_tick_params(pad=20)
plt.xlabel('Longitudes',fontsize=0.9*labelfont,labelpad=5)
ymajorLocator   = mpl.dates.WeekdayLocator(byweekday=2)
ymajorFormatter = mpl.dates.DateFormatter('%Y-%m-%d')
ax1.yaxis.set_major_locator(ymajorLocator)
ax1.yaxis.set_major_formatter(ymajorFormatter)
plt.yticks(fontsize=0.9*labelfont,rotation=0)
ax1.axes.yaxis.set_tick_params(pad=20) 
plt.title(labels[0], fontsize=1.2*labelfont,loc='left',pad=0.8*padspacescale, weight='bold',family='sans-serif')
ax1.spines['bottom'].set_linewidth(labelpadscale)
ax1.spines['left'].set_linewidth(labelpadscale)
ax1.spines['top'].set_linewidth(labelpadscale)
ax1.spines['right'].set_linewidth(labelpadscale)
plt.ylabel('Dates',fontsize=0.9*labelfont,labelpad=10)
plt.tick_params(axis='x',which='minor',bottom='on',left='on',top='on',right='on',length=10.0,width=8,colors='black',direction='in')
plt.tick_params(axis='y',which='major',bottom='on',left='on',top='on',right='on',length=10.0,width=8,colors='black',direction='in')
plt.tick_params(bottom='on',left='on',top='on',right='on',length=10.0,width=8,colors='black',direction='in')
cbaxes1  = fig.add_axes([0.02, 0.46, 0.4, 0.02])
cb1 = plt.colorbar(CB1,orientation='horizontal',cax = cbaxes1)
cb1.set_ticks(tick_marks1)
cb1.set_label(r'Sea level anomalies (m)', fontsize=0.9*labelfont,labelpad=0)
cb1.ax.tick_params(labelsize=0.9*labelfont)
ax1.set_position([l, b, 0.85*scale1*w, 1.9*scale1*h])
##############################################################################################
ax2        = fig.add_subplot(gs[1],facecolor='0.97')        
l, b, w, h = ax2.get_position().bounds
legends    = ['PK','BRAN_PK','AVISO_PK','AVISO']
ax2.plot(daily_dates2, PK_tide,         color='xkcd:black', linewidth=linefont)
ax2.plot(daily_dates2, SLA_PK_BRAN_ts,  color='dodgerblue', linewidth=1.0*linefont,linestyle='solid')
ax2.plot(daily_dates2, SLA_PK_AVISO_ts, color='xkcd:orange', linewidth=1.0*linefont,linestyle='solid')
ax2.plot(daily_dates2, SLA_AVISO_ts,    color='xkcd:green', linewidth=1.0*linefont,linestyle='solid')
plt.legend(legends, loc = 1, ncol = 4,  fontsize=18, facecolor='xkcd:pale blue', frameon=True, framealpha=0.8)
ax2.plot(daily_dates2, PK_tide*0,       color='xkcd:black', linewidth=1.0*linefont,linestyle='dashed')
plt.fill_between(time_daily[13:40], -0.25, 0.25, facecolor='xkcd:cool grey', alpha=0.4)
plt.fill_between(time_daily[60:88], -0.25, 0.25, facecolor='xkcd:cool grey', alpha=0.4)
plt.title(labels[1], fontsize=1.2*labelfont,loc='left', pad=0.8*padspacescale, weight='bold',family='sans-serif')
plt.xlim(daily_dates[0], daily_dates[-1])
plt.ylim(-0.25,0.25)
xmajorLocator   = mpl.dates.WeekdayLocator(byweekday=2,interval=1)
xmajorFormatter = mpl.dates.DateFormatter('%Y-%m-%d')
yminorLocator   = plt.MultipleLocator(0.05)
ax2.xaxis.set_minor_locator(xminorLocator)
ax2.xaxis.set_major_locator(xmajorLocator)
ax2.xaxis.set_major_formatter(xmajorFormatter)
ax2.axes.xaxis.set_tick_params(pad=labelpadscale)
ax2.yaxis.set_minor_locator(yminorLocator)
ax2.axes.yaxis.set_tick_params(pad=labelpadscale)
ax2.spines['bottom'].set_linewidth(labelpadscale)
ax2.spines['left'].set_linewidth(labelpadscale)
ax2.spines['top'].set_linewidth(labelpadscale)
ax2.spines['right'].set_linewidth(labelpadscale)
ax2.axes.xaxis.set_tick_params(pad=0.8*labelpadscale)
plt.grid(color ="tab:gray",linestyle='dashed', linewidth=0.4*linefont)
plt.xticks(fontsize=0.9*labelfont, rotation=75)
plt.xlabel('Date', fontsize=0.9*labelfont, labelpad=5)
plt.ylabel(r'Sea level anomalies (m)', fontsize=0.9*labelfont, labelpad=0)
plt.yticks(np.linspace(-0.2,0.2,5), ['-0.2','-0.1', '0.0', '0.1', '0.2'],fontsize=0.9*labelfont)
ax2.tick_params(axis='x',which='major',bottom='on',left='on',top='on',right='on',length=15.0,width=5,colors='black',direction='in')
ax2.tick_params(axis='y',which='minor',bottom='on',left='on',top='on',right='on',length=8.0,width=5,colors='black',direction='in')
ax2.tick_params(axis='y',which='major',left='on',length=15.0,width=5,colors='black',direction='in')
ax2.spines['left'].set_color('xkcd:black')
ax2.set_position([l-0.06, b+0.48, 1.0*scale1*w, 0.52*scale1*h])
plt.yticks(fontsize=labelfont)
##############################################################################################
ax3         = fig.add_subplot(gs[3])
l, b, w, h  = ax3.get_position().bounds
m           = Basemap(projection='merc',llcrnrlat=-38.0-0.001,urcrnrlat=-28+0.001,llcrnrlon=150.0-0.001,urcrnrlon=160-0.001,resolution='i')
lon2        = np.array([lon[22,20],lon[60,20],lon[60,33],lon[30,33],lon[22,20]])
lat2        = np.array([lat[22,20],lat[60,20],lat[60,33],lat[30,33],lon[22,20]])
lon3        = np.array([lon[22,20],lon[25,20],lon[33,33],lon[30,33],lon[22,20]])
lat3        = np.array([lat[22,20],lat[25,20],lat[33,33],lat[30,33],lat[22,20]])
x1, y1      = m(lon,lat)
x2, y2      = m(lon2,lat2)
x3, y3      = m(lon3,lat3)
xx          = np.arange(1, x1.shape[0], 2)
yy          = np.arange(1, y1.shape[1], 2)
points      = np.meshgrid(yy, xx)
point_index = tuple(points)
stipple_p   = SLA_MHW_pvalue
point_x     = x1[point_index]
point_y     = y1[point_index]
point_z     = stipple_p[point_index]
point_x     = np.ravel(point_x)
point_y     = np.ravel(point_y)
point_z     = np.ravel(point_z)
point_x     = point_x[~np.isnan(point_z)]
point_y     = point_y[~np.isnan(point_z)]      
CB1     = m.contourf(x1, y1, SLA_MHW,  cmap=cmaps1,levels=levels2,origin='lower',extend='both') 
CS1     = m.contour(x1,  y1, SSH_MHW,  ssh_levels1,linewidths=1.2*linefont,linestyles='solid',colors='black')
CS2     = m.contour(x1,  y1, SSH_mean, ssh_levels1,linewidths=1.2*linefont,linestyles='solid',colors='gray')
CS4     = m.contour(x1,  y1, depth,    np.arange(-201,-199,10), linewidths=1.2*linefont,linestyles='dashed',colors='xkcd:purple')
CS5     = m.scatter(point_x, point_y,  s = m_scale, c='dimgray', marker='.') 
m.drawcoastlines(color='0.1',  linewidth=0.5*linefont)
m.drawmapboundary(color='0.1', linewidth=0.5*linefont)
m.fillcontinents(color='0.95', lake_color='white')
m.plot(x3, y3, linewidth=1.2*linefont, linestyle='dashed', color='green')
m.drawparallels(np.arange(-40,-10, 2),labels=[1,0,0,0],linewidth=0.4*linefont,dashes=[2,2],color='gray',fontsize=0.9*labelfont)
m.drawmeridians(np.arange(150,165, 2),labels=[0,0,0,1],linewidth=0.4*linefont,dashes=[2,2],color='.7',fontsize=0.9*labelfont)  
plt.xlabel('Longitude',fontsize=0.9*labelfont,labelpad=0.2*padspacescale,family='sans-serif')
plt.ylabel('Latitude',fontsize=0.9*labelfont,labelpad=2*padspacescale,family='sans-serif')
plt.tick_params(axis='both',which='major',bottom='on',left='on',length=40.0,width=15,colors='black',direction='out')
plt.title(labels[2], fontsize=1.2*labelfont,loc='left',pad=0.6*padspacescale, weight='bold',family='sans-serif')
ax3.spines['bottom'].set_linewidth(labelpadscale)
ax3.spines['left'].set_linewidth(labelpadscale)
ax3.spines['top'].set_linewidth(labelpadscale)
ax3.spines['right'].set_linewidth(labelpadscale)
ax3.set_position([l-0.045, b+0.425, 1.0*scale1*w, 1.0*scale1*h])
cbaxes1     = fig.add_axes([0.53, 0.46, 0.3, 0.02])
cb1 = plt.colorbar(CB1,orientation='horizontal',cax = cbaxes1)
cb1.set_ticks(tick_marks2)
cb1.set_label(r'Sea level anomalies (m)', fontsize=0.9*labelfont,labelpad=0)
cb1.ax.tick_params(labelsize=0.9*labelfont)
fig.savefig(directory+'/MHW/Figure_plots/Figure6_Hovmoller_weekly_SLA_extreme.png',dpi=300,bbox_inches = 'tight')